[![AWS Data Wrangler](_static/logo.png "AWS Data Wrangler")](https://github.com/awslabs/aws-data-wrangler)

# 9 - Redshift - Append, Overwrite and Upsert

Wrangler's `copy_to_redshift` function has three different `mode` options.

1 - `append`

2 - `overwrite`

3 - `upsert`

In [1]:
import awswrangler as wr
import pandas as pd
from datetime import date

engine = wr.catalog.get_engine("aws-data-wrangler-redshift")

## Enter your bucket name:

In [2]:
import getpass
bucket = getpass.getpass()
path = f"s3://{bucket}/stage/"

 ··········································


## Enter your IAM ROLE ARN:

In [3]:
iam_role = getpass.getpass()

 ···············································································


### Creating the table (Overwriting if it exists)

In [4]:
df = pd.DataFrame({
    "id": [1, 2],
    "value": ["foo", "boo"],
    "date": [date(2020, 1, 1), date(2020, 1, 2)]
})

wr.db.copy_to_redshift(
    df=df,
    path=path,
    con=engine,
    schema="public",
    table="my_table",
    mode="overwrite",
    iam_role=iam_role,
    primary_keys=["id"]
)

wr.db.read_sql_table(table="my_table", schema="public", con=engine)

,id,value,date
0,2,boo,2020-01-02
1,1,foo,2020-01-01


## Appending

In [5]:
df = pd.DataFrame({
    "id": [3],
    "value": ["bar"],
    "date": [date(2020, 1, 3)]
})

wr.db.copy_to_redshift(
    df=df,
    path=path,
    con=engine,
    schema="public",
    table="my_table",
    mode="append",
    iam_role=iam_role,
    primary_keys=["id"]
)

wr.db.read_sql_table(table="my_table", schema="public", con=engine)

,id,value,date
0,2,boo,2020-01-02
1,1,foo,2020-01-01
2,3,bar,2020-01-03


## Upserting

In [6]:
df = pd.DataFrame({
    "id": [2, 3],
    "value": ["xoo", "bar"],
    "date": [date(2020, 1, 2), date(2020, 1, 3)]
})

wr.db.copy_to_redshift(
    df=df,
    path=path,
    con=engine,
    schema="public",
    table="my_table",
    mode="upsert",
    iam_role=iam_role,
    primary_keys=["id"]
)

wr.db.read_sql_table(table="my_table", schema="public", con=engine)

,id,value,date
0,3,bar,2020-01-03
1,1,foo,2020-01-01
2,2,xoo,2020-01-02


## Cleaning Up

In [7]:
with engine.connect() as con:
    con.execute("DROP TABLE public.my_table")